In [1]:
cd C:\Users\jschlajo\Desktop

C:\Users\jschlajo\Desktop


In [2]:
import pandas as pd

In [3]:
pos = pd.read_csv('player_position.csv')

In [4]:
pos.head()

,PLAYER_NAME,PLAYER_ID,PLAYER_POSITION
0,Jarell Martin,1626185,F
1,Amile Jefferson,1628518,F
2,Courtney Lee,201584,G
3,Ryan Arcidiacono,1627853,G
4,John Jenkins,203098,G


In [5]:
pos = pos.drop_duplicates()
pos = pos[['PLAYER_ID','PLAYER_POSITION']]
pos['PLAYER_POSITION'].fillna('N', inplace = True)
pos['PLAYER_POSITION'] = pos['PLAYER_POSITION'].map(lambda x : x.split('-')[0])

In [6]:
nba = pd.read_csv('nba_stats.csv')
nba = nba.drop('Unnamed: 0', axis=1)
nba['GAME_DATE'] = nba['GAME_DATE'].map(lambda x : x.split('T')[0])

nba.head()

,SEASON_YEAR,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,...,STL_RANK,BLK_RANK,BLKA_RANK,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK,NBA_FANTASY_PTS_RANK,DD2_RANK,TD3_RANK
0,2018-19,201961,Wayne Ellington,1610612765,DET,Detroit Pistons,21801152,2019-04-01,DET @ IND,L,...,11467,2443,1,16060,4640,1215,11380,3530,2210,116
1,2018-19,203081,Damian Lillard,1610612757,POR,Portland Trail Blazers,21801157,2019-04-01,POR @ MIN,W,...,11467,7843,16167,10439,7537,5967,8692,4510,1,116
2,2018-19,203482,Kelly Olynyk,1610612748,MIA,Miami Heat,21801153,2019-04-01,MIA @ BOS,L,...,1393,2443,1,4536,4640,6765,12440,6816,2210,116
3,2018-19,202720,Jon Leuer,1610612765,DET,Detroit Pistons,21801152,2019-04-01,DET @ IND,L,...,11467,7843,16167,1,17837,22077,20009,23796,2210,116
4,2018-19,203967,Dario Saric,1610612750,MIN,Minnesota Timberwolves,21801157,2019-04-01,MIN vs. POR,L,...,11467,7843,16167,10439,11948,7584,11380,12911,2210,116


In [15]:
nba = nba.merge(pos, left_on='PLAYER_ID', right_on='PLAYER_ID', how='left')

In [29]:
from sqlalchemy import create_engine

disk_engine = create_engine('sqlite:///db/nba_stats.sqlite')
nba.to_sql('nba', disk_engine, if_exists='append', index=False)

In [7]:
cd "C:\Users\jschlajo\Desktop\project_two\

C:\Users\jschlajo\Desktop\project_two


In [ ]:
import os
import pandas as pd
import numpy as np
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from flask import Flask, jsonify, render_template
from flask_sqlalchemy import SQLAlchemy

app = Flask(__name__)
disk_engine = create_engine('sqlite:///db/nba_stats.sqlite')


@app.route("/")
def index():


    return render_template("index.html")

# render_template("index.html", table = table)

@app.route("/metadata/date")
def sample_metadata_1():
    """Return the MetaData for a given sample."""
    
    df = pd.read_sql_query(f'SELECT DISTINCT(GAME_DATE) FROM nba',disk_engine)
    date_dict = df.to_dict()
    print(sample_metadata_1)
    return jsonify(date_dict)


@app.route("/metadata/<date_data>")
def sample_metadata(date_data):
    """Return the MetaData for a given sample."""
    
    df = pd.read_sql_query(f'SELECT GAME_DATE, TEAM_ABBREVIATION, TEAM_NAME, MATCHUP,\
    PLAYER_NAME, PLAYER_POSITION,NBA_FANTASY_PTS, FG3M, AST, BLK, FGM, FTM, REB, STL, TOV FROM nba WHERE GAME_DATE = "{date_data}"', disk_engine)
    column_order = ['GAME_DATE', 'TEAM_ABBREVIATION', 'TEAM_NAME', 'MATCHUP','PLAYER_NAME', 'PLAYER_POSITION','NBA_FANTASY_PTS', 'FG3M', 'AST', 'BLK', 'FGM', 'FTM', 'REB', 'STL', 'TOV']
    df.columns = ['GAME_DATE', 'TEAM_ABBR', 'TEAM_NAME', 'MATCHUP','PLAYER_NAME', 'PLAYER_POSITION','NBA_FANTASY_PTS', 'FG3M', 'AST', 'BLK', 'FGM', 'FTM', 'REB', 'STL', 'TOV']

    d = df.to_dict('records')

    container = []
    for x in d:
        resorted = sorted(x.items(), key=lambda i:column_order.index(i[0]))
        container.append(resorted)


    print(sample_metadata)
    return jsonify(container)


@app.route("/metadata/all_data")
def all_metadata():
    """Return the MetaData for a given sample."""
    
    df = pd.read_sql_query(f'SELECT * FROM nba',disk_engine)
    data = df.to_dict('records')
    print(sample_metadata)
    return jsonify(data)


if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [11/Apr/2019 21:02:36] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2019 21:02:36] "GET /style.css HTTP/1.1" 404 -
127.0.0.1 - - [11/Apr/2019 21:02:36] "GET /images/airjordan.gif HTTP/1.1" 404 -
127.0.0.1 - - [11/Apr/2019 21:02:36] "GET /images/coolstory.gif HTTP/1.1" 404 -
127.0.0.1 - - [11/Apr/2019 21:02:36] "GET /config.js HTTP/1.1" 404 -
127.0.0.1 - - [11/Apr/2019 21:02:36] "GET /static/js/bonus.js HTTP/1.1" 404 -
127.0.0.1 - - [11/Apr/2019 21:02:36] "GET /static/js/app.js HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2019 21:02:36] "GET /images/lebron.gif HTTP/1.1" 404 -
127.0.0.1 - - [11/Apr/2019 21:02:36] "GET /images/logo.png HTTP/1.1" 404 -
127.0.0.1 - - [11/Apr/2019 21:02:39] "GET /static/js/bonus.js HTTP/1.1" 404 -
127.0.0.1 - - [11/Apr/2019 21:02:39] "GET /metadata/date HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2019 21:02:39] "GET /config.js HTTP/1.1" 404 -
127.0.0.1 - - [11/Apr/2019 21:02:39] "GET /favicon.ico H

<function sample_metadata_1 at 0x000001F270718488>


[2019-04-11 21:02:42,517] ERROR in app: Exception on /metadata/2019-03-20 [GET]
Traceback (most recent call last):
  File "C:\Users\jschlajo\AppData\Local\Continuum\anaconda3\lib\site-packages\flask\app.py", line 2292, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\jschlajo\AppData\Local\Continuum\anaconda3\lib\site-packages\flask\app.py", line 1815, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\jschlajo\AppData\Local\Continuum\anaconda3\lib\site-packages\flask\app.py", line 1718, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\jschlajo\AppData\Local\Continuum\anaconda3\lib\site-packages\flask\_compat.py", line 35, in reraise
    raise value
  File "C:\Users\jschlajo\AppData\Local\Continuum\anaconda3\lib\site-packages\flask\app.py", line 1813, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\jschlajo\AppData\Local\Continuum\anaconda3\lib\site-packages\flask\app.py", lin